In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
from ipywidgets import Button, VBox, HBox, Output, Label
from IPython.display import display
import glob
from scipy import signal
from matplotlib import cm

# Making Predicitons of Seismic Data




## Level 1 of Filtering the data

### Adjusting settings to short list potential events

In [ ]:
# Define settings for Moon and Mars
settings = {
    'moon': {
        'low_freq_point': 0.2,#0.6
        'high_freq_point': 1,
        'frequence_window': 0.2,
        'sta_len': 100,
        'lta_len': 1000,
        'thr_on': 3,
        'resample': False
    },
    'mars': {
        'low_freq_point': 0.6,
        'high_freq_point': 4,
        'frequence_window': 0.5,
        'sta_len': 20,
        'lta_len': 80,
        'thr_on': 2.1,
        'resample': True
    }
}

# Function to get settings based on event type
def get_settings(event_type):
    return settings.get(event_type, settings['moon'])  # Default to 'moon' if event_type is not found

### Selecting the Files to make predictions

In [ ]:
#mseed_files = sorted(glob.glob('./data/mars/training/data/*.mseed'))
#mseed_files = sorted(glob.glob('./data/mars/test/data/*.mseed'))

mseed_files = sorted(glob.glob('./data/lunar/training/data/S12_GradeA/*.mseed'))
#mseed_files = sorted(glob.glob('./data/lunar/test/data/S12_GradeB/*.mseed'))
#mseed_files = sorted(glob.glob('./data/lunar/test/data/S15_GradeA/*.mseed'))
#mseed_files = sorted(glob.glob('./data/lunar/test/data/S15_GradeB/*.mseed'))
#mseed_files = sorted(glob.glob('./data/lunar/test/data/S16_GradeA/*.mseed'))
#mseed_files = sorted(glob.glob('./data/lunar/test/data/S16_GradeB/*.mseed'))

event_info = []
filename = ''


# Determine event type based on the file path
if 'lunar' in mseed_files[0]:
    event_type = 'moon'
elif 'mars' in mseed_files[0]:
    event_type = 'mars'
else:
    event_type = 'moon'  # Default to 'moon' if neither is found

# Get settings based on the determined event type
current_settings = get_settings(event_type)
plot = True

### Essential Functions definations
#### calculating best frequency range to filter data with

In [ ]:
def load_mseed(file_idx):
    st = read(mseed_files[file_idx])
    return st, mseed_files[file_idx]



def calculate_best_freq_range_by_std(st, resample):
    # Define frequency ranges for filtering
    freq_ranges = [
          (start, start + current_settings['frequence_window']) 
          for start in np.arange(
              current_settings['low_freq_point'], 
              current_settings['high_freq_point'], 
              0.1)]
    
    stddev_in_ranges = []
    for f_min, f_max in freq_ranges:
        # Copy the original trace for filtering
        st_filt = st.copy()
        # Apply bandpass filter for the given frequency range
        st_filt.filter('bandpass', freqmin=f_min, freqmax=f_max)
        # Extract the filtered trace data
        tr_filt = st_filt.traces[0].copy()
        #if(resample):
            #tr_filt.resample(6.625)
        tr_data_filt = tr_filt.data
        # Normalize tr_data_filt between -1 and 1
        tr_data_filt = 2 * (tr_data_filt - np.min(tr_data_filt)) / (np.max(tr_data_filt) - np.min(tr_data_filt)) - 1
        
        # Calculate standard deviation of the filtered signal
        stddev = np.std(tr_data_filt)
        stddev_in_ranges.append((f_min, f_max, stddev))
    # Find the frequency range with the minimum standard deviation
    best_range = min(stddev_in_ranges, key=lambda x: x[2])

    print(f"The best frequency range is {best_range[0]:.1f} to {best_range[1]:.1f} with minimum standard deviation {best_range[2]:.3f}")
    return best_range 

def calculate_best_freq_range_by_pwr(st, resample):
    # Extract the trace data and sampling rate
    tr = st.traces[0].copy()
    sampling_rate = tr.stats.sampling_rate
    tr_data = tr.data

    # Define frequency ranges for filtering
    freq_ranges = [
        (start, start + current_settings['frequence_window']) 
        for start in np.arange(
            current_settings['low_freq_point'], 
            current_settings['high_freq_point'],
            0.1
        )
    ]
    # Initialize list to store top 1000 power averages for each frequency range
    max_power_in_ranges = []

    # Loop through each frequency range
    for f_min, f_max in freq_ranges:
        # Copy the original trace for filtering
        st_filt = st.copy()

        # Apply bandpass filter for the given frequency range
        st_filt.filter('bandpass', freqmin=f_min, freqmax=f_max)

        # Extract the filtered trace data
        tr_filt = st_filt.traces[0].data

        # Compute the spectrogram for the filtered signal
        fu, tu, sxxu = signal.spectrogram(tr_filt, fs=sampling_rate)
        sxxu_dB = 10 * np.log10(sxxu)

        # Flatten the entire spectrogram (all frequencies and time points)
        power_values_flat = sxxu_dB.flatten()

        # Sort the power values and take the top 1000
        top_1000_power_values = np.sort(power_values_flat)[-50:]  # Top 1000 largest values

        # Calculate the average of the top 1000 values
        top_1000_avg_power = np.mean(top_1000_power_values)
        print(top_1000_avg_power)

        # Store the frequency range and the calculated average power
        max_power_in_ranges.append((f_min, f_max, top_1000_avg_power))

    # Find the frequency range with the greatest average of the top 1000 power values
    best_range = max(max_power_in_ranges, key=lambda x: x[2])

    print(f"The best frequency range is {best_range[0]:.1f} to {best_range[1]:.1f} with average power {best_range[2]:.3f}")
    return best_range




### Detecting Events according to 1st Filter Algorithms

In [ ]:
current_file_idx = 0    
def detect_events(max_files = 100):
    num_files_to_display = len(mseed_files)
    if(num_files_to_display > max_files):
        num_files_to_display = max_files

    print('Detecting events.....')
    print('Event Type is :', event_type)
    if(plot):
        # Create a vertical plot layout based on the number of files (each file gets 4 plots)
        fig, axs = plt.subplots(num_files_to_display, 2, figsize=(20, 5 * num_files_to_display))
    resample = current_settings['resample']
    event_number = 0
    for i in range(num_files_to_display):
        global filename
        # Load the MiniSEED file
        st, filename = load_mseed(current_file_idx + i)  # Incrementing file index
        # Extract the evid part from the filename
        evid = filename.split('_')[-1].split('.')[0]
        filename = filename
        
        
        #best_range = calculate_best_freq_range_by_std(st, resample)
        best_range = calculate_best_freq_range_by_pwr(st, resample)
        # Define the frequency range for the bandpass filter
        # minfreq = 0.8
        # maxfreq = 0.9
        minfreq = best_range[0]
        maxfreq = best_range[1]

        # ============================Apply the filter
        st_filt = st.copy()
        st_filt.filter('bandpass', freqmin=minfreq, freqmax=maxfreq)
        #st_filt.filter('bandpass', freqmin=3, freqmax=4)
        if(resample):
            st_filt.resample(6.625)
        tr_filt = st_filt.traces[0].copy()
        tr_times_filt = tr_filt.times()
        tr_data_filt = tr_filt.data
        stddev = np.std(tr_data_filt)
        threshold_upper = 26 * stddev
        threshold_lower = -26 * stddev
        tr_data_filt[np.abs(tr_data_filt) > threshold_upper] = stddev 
        tr_data_filt[np.abs(tr_data_filt) < threshold_lower] = -stddev
        # Normalize tr_data_filt between -1 and 1
        tr_data_filt = 2 * (tr_data_filt - np.min(tr_data_filt)) / (np.max(tr_data_filt) - np.min(tr_data_filt)) - 1

        from obspy.signal.trigger import classic_sta_lta, plot_trigger, trigger_onset

        sta_len = current_settings['sta_len']
        lta_len = current_settings['lta_len']

        tr_data_abs = np.abs(tr_data_filt)
        df = 6.625
        # Run Obspy's STA/LTA to obtain a characteristic function
        cft = classic_sta_lta(tr_data_abs, int(sta_len * df), int(lta_len * df))
        average_cft = np.mean(cft)
        print(f'Average CFT: {average_cft}')

        thr_on = current_settings['thr_on']
        thr_off = average_cft
        on_off = np.array(trigger_onset(cft, thr_on, thr_off))


        cat_directory = './data/lunar/training/catalogs/'
        cat_file = cat_directory + 'apollo12_catalog_GradeA_final.csv'
        cat = pd.read_csv(cat_file)
        row = cat.iloc[i]
        arrival_time_rel = row['time_rel(sec)']

        if(plot):
            # Plot filtered trace in the first column
            axs[i, 0].plot(tr_times_filt, tr_data_filt)
            axs[i, 0].set_xlim([min(tr_times_filt), max(tr_times_filt)])
            axs[i, 0].set_ylabel('Velocity (m/s)')
            axs[i, 0].set_xlabel('Time (s)')
            axs[i, 0].set_title(f'Filtered Time Series of {evid} with {i}')
            arrival_line = axs[i, 0].axvline(x=arrival_time_rel, c='purple', label='Abs. Arrival')
            axs[i, 0].annotate(f'Arrival TIme at {arrival_time_rel}', xy=(arrival_time_rel, tr_data_filt.max()), 
                            xytext=(arrival_time_rel + 0.1, tr_data_filt.max()+0.1),  # Adjust text position
                            color='black', fontsize=11)
            for j in np.arange(0, len(on_off)):
                triggers = on_off[j]
                if(triggers[1] - triggers[0] > 1200) or (len(on_off) < 50):
                    event_number += 1
                    exact_filename = filename.split('\\')[-1].rsplit('.', 1)[0]
                    event_info.append([exact_filename, event_number, f'{i} | {j}', (round(tr_times_filt[triggers[0]]-100)), 0, round(best_range[0],1), round(best_range[1],1),  ])
                    # Draw the vertical line
                    axs[i, 0].axvline(x=tr_times_filt[triggers[0]], color='red', label=f'Trig. On {j+1}')
                    #print(f'Trig. On {i+1} of {j+1} at {triggers[0]}')
                    # Annotate with the trigger number (or any custom label)
                    dj = (j - 1) % 16 + 1
                    axs[i, 0].annotate(f'Trig {j} at {round(tr_times_filt[triggers[0]])}s', xy=(tr_times_filt[triggers[0]], tr_data_filt.max()), 
                                    xytext=(tr_times_filt[triggers[0]] + 0.1, tr_data_filt.max()-(dj/8)),  # Adjust text position
                                    color='grey', fontsize=11)
            # Plot characteristic function
            axs[i,1].plot(tr_times_filt,cft)
            axs[i,1].set_xlim([min(tr_times_filt),max(tr_times_filt)])
            axs[i,1].set_xlabel('Time (s)')
            axs[i,1].set_ylabel('Characteristic function')

        
    if(plot):
        # Adjust layout for better spacing
        plt.tight_layout()
        plt.show()

### initializing event detection for 1st level filteration
#### select how many to filter in the folder

In [ ]:

detect_events(max_files = 100)

### Saving 1st filter results in catalogue

In [ ]:
folder_name = filename.split('/')[-1].split('\\')[0]
# Convert the event information list to a DataFrame
event_df = pd.DataFrame(event_info, columns=['filename', 'Event Number','events_in_day(day|event)', 'time_rel(sec)', 'time_abs(sec)', 'Filter_low_freq_point', 'Filter_high_freq_point'])
# Define the folder path
folder_path = f'initial_filterations_catalogue/{event_type}'
# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)
savePath = f'{folder_path}/initial_filtered_events_of_{folder_name}.csv'
event_df.to_csv(savePath, index=False)

### Reading Catalogue files and making numpy arrays according to them
#### for CNN to make final Prediciton

In [ ]:
from obspy.signal.filter import bandpass

# Load the CSV file
csv_file = savePath
df = pd.read_csv(csv_file)

# Create empty lists to store the data and labels
data_list = []
labels = []
aux_data_list = []  # To store auxiliary data (std dev before and after)

# Define the fixed length for the data (e.g., 14 minutes of data at a specific sampling rate)
fixed_length = 5565  # Example: 14 minutes of data at 100 Hz sampling rate

# Loop through each row in the CSV file
for idx, row in df.iterrows():
    filename = row['filename']
    time_rel = row['time_rel(sec)']  # Relative time point in seconds
    filter_low = row['Filter_low_freq_point']  # Low cut-off frequency
    filter_high = row['Filter_high_freq_point']  # High cut-off frequency
    label = row['label']  # Event label (0 or 1)

    # Load the MiniSEED file
    data_directory = './data/lunar/training/data/S12_GradeA/'
    mseed_file = f'{data_directory}{filename}.mseed'
    st = read(mseed_file)

    # Apply bandpass filter (between filter_low and filter_high)
    filtered_st = st.filter('bandpass', freqmin=filter_low, freqmax=filter_high)

    # Define the start and end times for slicing the data (4 minutes before and 10 minutes after time_rel)
    start_time = time_rel - 4 * 60  # 4 minutes before
    end_time = time_rel + 10 * 60  # 10 minutes after

    # Cut the data from start_time to end_time
    sliced_st = filtered_st.slice(starttime=st[0].stats.starttime + start_time, 
                                  endtime=st[0].stats.starttime + end_time)

    # Normalize the sliced data (mean=0, std=1)
    sliced_data = sliced_st[0].data
    sliced_data_normalized = (sliced_data - np.mean(sliced_data)) / np.std(sliced_data)

    # Calculate standard deviation before and after time_rel
    before_data = filtered_st.slice(starttime=st[0].stats.starttime + start_time, 
                                     endtime=st[0].stats.starttime + time_rel)
    after_data = filtered_st.slice(starttime=st[0].stats.starttime + time_rel, 
                                    endtime=st[0].stats.starttime + end_time)
    
    std_before = np.std(before_data[0].data)
    std_after = np.std(after_data[0].data)

    # Pad or truncate the data to the fixed length
    if len(sliced_data_normalized) > fixed_length:
        sliced_data_normalized = sliced_data_normalized[:fixed_length]
    else:
        sliced_data_normalized = np.pad(sliced_data_normalized, (0, fixed_length - len(sliced_data_normalized)), 'constant')

    # Store the normalized data, labels, and auxiliary data
    data_list.append(sliced_data_normalized)
    labels.append(label)
    aux_data_list.append([std_before, std_after])  # Append standard deviations

# Convert lists to NumPy arrays
X_data = np.array(data_list)
y_labels = np.array(labels)
aux_data = np.array(aux_data_list)  # Auxiliary data containing std devs

# Now, X_data contains the processed seismogram data,
# y_labels contains the labels, and aux_data contains the std devs
print(f"Data shape: {X_data.shape}, Labels shape: {y_labels.shape}, Auxiliary data shape: {aux_data.shape}")
